In [ ]:
import itertools
import math
import os
import random
import shutil
from pathlib import Path
from typing import Literal, cast

import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
# from tensorflow.keras.layers.experimental import preprocessing

### Parameters

In [ ]:
# https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/#:~:text=For%20B0%20to%20B7%20base%20models%2C%20the%20input%20shapes%20are%20different.%20Here%20is%20a%20list%20of%20input%20shape%20expected%20for%20each%20model%3A
# IMAGE_SIZE = 224  # MobileNet
# IMAGE_SIZE = 300  # EfficientNetB3
IMAGE_SIZE = 456  # EfficientNetB5
# IMAGE_SIZE = 600  # EfficientNetB7

# BATCH_SIZE = 32
BATCH_SIZE = 16

TARGET_SIZE = (IMAGE_SIZE, IMAGE_SIZE)
CHANNELS = 3
INPUT_SHAPE = (TARGET_SIZE[0], TARGET_SIZE[1], CHANNELS)


RANDOM_SEED = 123

dataset_input_path, dataset_output_path = "../../data", "../../data"
# dataset_input_path, dataset_output_path = "/kaggle/input/sea-animals-image-dataste", "/kaggle/working/"

dataset_path = dataset_output_path

### Create directories /train, /valid and /test

In [ ]:
def directory_is_empty(directory: Path) -> bool:
    if not os.path.isdir(directory):
        return True
    return len(os.listdir(directory)) == 0


def calculate_split_points(images_count: int, train_ratio: float, valid_ratio: float) -> tuple[int, int]:
    train_valid_split_point = math.ceil(images_count * train_ratio)
    valid_test_split_point = train_valid_split_point + math.floor(images_count * valid_ratio)
    return train_valid_split_point, valid_test_split_point


def create_train_test_subdirectories(train_ratio: float = 0.7, validation_ratio: float = 0.1) -> None:
    data_input_path = Path(dataset_input_path)  # Has one subdirectory for each Animal
    if not data_input_path.is_dir():
        raise ValueError(f"Path '{data_input_path}' is not a directory.")
    data_output_path = Path(dataset_output_path)  # Has one subdirectory for each Animal
    train_path = data_output_path / "train"
    valid_path = data_output_path / "valid"
    test_path = data_output_path / "test"
    if not directory_is_empty(train_path) and not directory_is_empty(test_path) and not directory_is_empty(valid_path):
        print("/train, /valid and /test directories already exist!")
        return
    for animal_dir in data_input_path.iterdir():
        if not animal_dir.is_dir() or animal_dir.name in ["train", "valid", "test"]:
            continue
        animal_images = [i for i in animal_dir.glob("*") if i.is_file()]
        random.seed(RANDOM_SEED)
        random.shuffle(animal_images)
        train_valid_split_point, valid_test_split_point = calculate_split_points(
            len(animal_images),
            train_ratio,
            validation_ratio,
        )
        train_images = animal_images[:train_valid_split_point]
        valid_images = animal_images[train_valid_split_point:valid_test_split_point]
        test_images = animal_images[valid_test_split_point:]
        train_animal_path = train_path / animal_dir.name
        valid_animal_path = valid_path / animal_dir.name
        test_animal_path = test_path / animal_dir.name
        train_animal_path.mkdir(parents=True, exist_ok=True)
        valid_animal_path.mkdir(parents=True, exist_ok=True)
        test_animal_path.mkdir(parents=True, exist_ok=True)
        for train_image in train_images:
            shutil.copy(str(train_image), str(train_animal_path / train_image.name))
        for valid_image in valid_images:
            shutil.copy(str(valid_image), str(valid_animal_path / valid_image.name))
        for test_image in test_images:
            shutil.copy(str(test_image), str(test_animal_path / test_image.name))
        print(f"{animal_dir.name}:")
        print(f"  - Moved {len(train_images)} images to 'train/{animal_dir.name}'")
        print(f"  - Moved {len(valid_images)} images to 'valid/{animal_dir.name}'")
        print(f"  - Moved {len(test_images)} images to 'test/{animal_dir.name}'")


create_train_test_subdirectories()

### Placing data into a Dataframe

In [ ]:
def get_images_dataframe(directory: Path) -> pd.DataFrame:
    extensions = ["JPG", "jpg", "png", "PNG"]
    images_by_extension = [list(directory.glob(rf"**/*.{extension}")) for extension in extensions]
    filepaths_with_duplicates = [
        image for images_for_extension in images_by_extension for image in images_for_extension
    ]
    filepaths = list(set(filepaths_with_duplicates))

    def get_label_for_path(path: Path) -> str:
        # data/<LABEL>/<fileName>
        path_without_filename = os.path.split(path)[0]
        label = os.path.split(path_without_filename)[1]
        return label

    labels = [get_label_for_path(x) for x in filepaths]

    filepaths = pd.Series(filepaths, name="Filepath").astype(str)
    labels = pd.Series(labels, name="Label")
    images_df = pd.concat([filepaths, labels], axis=1)
    return images_df


images_dir = Path(dataset_path)
train_dir = images_dir / "train"
valid_dir = images_dir / "valid"
test_dir = images_dir / "test"

train_df = get_images_dataframe(train_dir)
valid_df = get_images_dataframe(valid_dir)
test_df = get_images_dataframe(test_dir)
images_df = pd.concat([train_df, valid_df, test_df], ignore_index=True)
num_classes = images_df["Label"].drop_duplicates().count()

assert len(images_df) == 13711
assert num_classes == 23

print(f"{num_classes=}")
print(images_df)

### Visualizing images from the dataset

In [ ]:
def show_quantities_by_animal() -> None:
    label_counts = images_df["Label"].value_counts()

    fix, ax = plt.subplots(figsize=(20, 6))
    sns.barplot(x=label_counts.index, y=label_counts.values, alpha=0.8, palette="rocket")
    ax.set_title("Cantidad de imagenes de cada animal en el dataset", fontsize=16)
    ax.set_xlabel("Animal", fontsize=14)
    ax.set_ylabel("Cantidad", fontsize=14)
    ax.tick_params(axis="x", labelrotation=45)
    plt.show()


show_quantities_by_animal()

In [ ]:
def show_random_examples() -> None:
    rows = 3
    columns = 3
    random_indexes = np.random.randint(0, len(images_df), rows * columns)
    fig, axes = plt.subplots(nrows=rows, ncols=columns, figsize=(5, 5), subplot_kw={"xticks": [], "yticks": []})

    for i, ax in enumerate(axes.flat):
        ax.imshow(plt.imread(images_df["Filepath"][random_indexes[i]]))
        ax.set_title(images_df["Label"][random_indexes[i]])
    plt.tight_layout()
    plt.show()


show_random_examples()

### Data Preprocessing
The data will be split into three different categories: Training, Validation and Testing.

In [ ]:
def read_image_dataset_from_directory(
    path: Path,
    validation_split: float | None = None,
    subset: Literal["training", "validation", "both"] | None = None,
    shuffle: bool = True,
) -> tf.data.Dataset:
    dataset = keras.utils.image_dataset_from_directory(
        directory=path,
        labels="inferred",
        label_mode="int",  # for `sparse_categorical_crossentropy` loss
        # label_mode="categorical",  # for `categorical_crossentropy` loss
        color_mode="rgb",
        batch_size=BATCH_SIZE,
        image_size=IMAGE_SIZE,
        shuffle=shuffle,
        seed=RANDOM_SEED,
        validation_split=validation_split,
        subset=subset,
    )
    dataset = cast(tf.data.Dataset, dataset)
    return dataset


def get_train_validate_test_images_dataset() -> tuple[tf.data.Dataset, tf.data.Dataset, tf.data.Dataset]:
    train_dataset = read_image_dataset_from_directory(train_dir)
    validation_dataset = read_image_dataset_from_directory(valid_dir)
    test_dataset = read_image_dataset_from_directory(test_dir, shuffle=False)
    return train_dataset, validation_dataset, test_dataset


train_dataset, validation_dataset, test_dataset = get_train_validate_test_images_dataset()

### Training the model

In [ ]:
def use_mobilenet_pretrained_model() -> keras.Model:
    mobile_model = keras.applications.MobileNetV2(
        input_shape=INPUT_SHAPE,
        include_top=False,
        weights="imagenet",
        # pooling="avg",
        # pooling="max",
        pooling=None,
    )
    mobile_model.trainable = False

    model = keras.Sequential()
    model.add(mobile_model)
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation="relu"))
    model.add(keras.layers.Dropout(0.45))
    model.add(keras.layers.Dense(256, activation="relu"))
    model.add(keras.layers.Dropout(0.45))
    model.add(keras.layers.Dense(num_classes, activation="softmax"))
    optimizer = keras.optimizers.Adam()
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model


def use_efficientnet_pretrained_model() -> keras.Model:
    # efficientnet_model = keras.applications.EfficientNetB3(
    efficientnet_model = keras.applications.EfficientNetB5(
        input_shape=INPUT_SHAPE,
        include_top=False,
        weights="imagenet",
        # pooling="avg",
        pooling="max",
        # pooling=None,
    )
    efficientnet_model.trainable = False

    data_augmentation = keras.Sequential(
        [
            keras.layers.RandomFlip("horizontal", input_shape=INPUT_SHAPE),
            keras.layers.RandomRotation(0.1),
            keras.layers.RandomZoom(0.1),
            # fill_mode="nearest",
        ]
    )
    inputs = keras.Input(shape=INPUT_SHAPE)
    x = data_augmentation(inputs)
    x = efficientnet_model(x, training=False)
    x = keras.layers.Dropout(0.3)(x)
    x = keras.layers.Dense(128, activation="relu")(x)
    outputs = keras.layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model(inputs, outputs)
    # optimizer = keras.optimizers.Adamax()
    optimizer = keras.optimizers.Adam()
    loss = keras.losses.SparseCategoricalCrossentropy()
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=["accuracy"],
    )
    return model

    # model = keras.Sequential()
    # model.add(efficientnet_model)
    # model.add(keras.layers.Flatten())
    # model.add(keras.layers.Dense(128, activation="relu"))
    # model.add(keras.layers.Dropout(0.45))
    # model.add(keras.layers.Dense(256, activation="relu"))
    # model.add(keras.layers.Dropout(0.45))
    # model.add(keras.layers.Dense(num_classes, activation="softmax"))
    # return model


def use_custom_model() -> keras.Model:
    model = keras.Sequential(
        [
            keras.layers.Input(shape=INPUT_SHAPE),
            keras.layers.Conv2D(32, (3, 3), activation="relu"),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Conv2D(64, (3, 3), activation="relu"),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Conv2D(128, (3, 3), activation="relu"),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Flatten(),
            keras.layers.Dense(128, activation="relu"),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    optimizer = keras.optimizers.Adam()
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model


model = use_efficientnet_pretrained_model()
model.summary()

In [ ]:
EPOCHS = 15
callbacks = []

history: keras.callbacks.History = model.fit(
    # train_images,
    train_dataset,
    epochs=EPOCHS,
    verbose=1,
    # steps_per_epoch=len(train_images),
    validation_data=validation_dataset,
    # validation_steps=len(validation_images),
    callbacks=callbacks,
    # shuffle=False,
)

In [ ]:
model.save_weights("animals_eff.weights.h5")

### Model Evaluation

In [ ]:
loss, accuracy = model.evaluate(test_dataset, verbose=0)

print("    Test Loss: {:.5f}".format(loss))
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
print(history.history.keys())

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

EPOCHS = range(len(accuracy))
plt.plot(EPOCHS, accuracy, "b", label="Training accuracy")
plt.plot(EPOCHS, val_accuracy, "r", label="Validation accuracy")

plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(EPOCHS, loss, "b", label="Training loss")
plt.plot(EPOCHS, val_loss, "r", label="Validation loss")

plt.title("Training and validation loss")
plt.legend()
plt.show()

### Making predictions on the Test Data

In [ ]:
# Predict the label of the test_images
pred = model.predict(test_dataset)
pred = np.argmax(pred, axis=1)

print(pred)

# Map the label
# labels = train_images.class_indices
# labels = dict((v, k) for k, v in labels.items())
# pred = [labels[k] for k in pred]

# Display the result
print(f"The first 5 predictions: {pred[:5]}")

In [ ]:
# Display 25 random pictures from the dataset with their labels
random_indexes = np.random.randint(0, len(test_df) - 1, 15)
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(25, 15), subplot_kw={"xticks": [], "yticks": []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_df.Filepath.iloc[random_indexes[i]]))
    if test_df.Label.iloc[random_indexes[i]] == pred[random_indexes[i]]:
        color = "green"
    else:
        color = "red"
    ax.set_title(f"True: {test_df.Label.iloc[random_indexes[i]]}\nPredicted: {pred[random_indexes[i]]}", color=color)
plt.show()
plt.tight_layout()

### Plotting the Classification Reports and Confusion Matrix

A <b>confusion matrix</b> is a table that summarizes the number of correct and incorrect predictions made by a classification model on a set of test data. It is usually represented as a square matrix with rows and columns representing the predicted and true class labels, respectively. The entries of the matrix indicate the number of test samples that belong to a certain class, and how many of those were classified correctly or incorrectly by the model. A confusion matrix can provide a detailed breakdown of the performance of the model, including measures such as accuracy, precision, recall, and F1-score for each class. It can be used to identify specific areas where the model is making errors, and to diagnose problems with the model's predictions.

A <b>classification report</b> is a summary of the key performance metrics for a classification model, including precision, recall, and F1-score, as well as the overall accuracy of the model. It provides a concise overview of the model's performance, typically broken down by class, and can be used to quickly assess the strengths and weaknesses of the model. The report is often presented as a table, with each row representing a class and columns showing various performance metrics. The report may also include other metrics such as support (the number of test samples belonging to a particular class), and the macro- and micro-averages of the performance metrics across all classes.

In [ ]:
y_test = list(test_df.Label)
print(classification_report(y_test, pred))

In [ ]:
report = classification_report(y_test, pred, output_dict=True)
df = pd.DataFrame(report).transpose()
df

In [ ]:
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(20, 14), text_size=10, norm=False, savefig=False):
    """Makes a labelled confusion matrix comparing predictions and ground truth labels.

      If classes is passed, confusion matrix will be labelled, if not, integer class values
    will be used.

    Args:
      y_true: Array of truth labels (must be same shape as y_pred).
      y_pred: Array of predicted labels (must be same shape as y_true).
      classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
      figsize: Size of output figure (default=(10, 10)).
      text_size: Size of output figure text (default=15).
      norm: normalize values or not (default=False).
      savefig: save confusion matrix to file (default=False).

    Returns:
      A labelled confusion matrix plot comparing y_true and y_pred.

    Example usage:
      make_confusion_matrix(y_true=test_labels, # ground truth test labels
                            y_pred=y_preds, # predicted labels
                            classes=class_names, # array of class label names
                            figsize=(15, 15),
                            text_size=10)
    """
    # Create the confustion matrix
    cm = confusion_matrix(y_true, y_pred)
    cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]  # normalize it
    n_classes = cm.shape[0]  # find the number of classes we're dealing with

    # Plot the figure and make it pretty
    fig, ax = plt.subplots(figsize=figsize)
    cax = ax.matshow(cm, cmap=plt.cm.Blues)  # colors will represent how 'correct' a class is, darker == better
    fig.colorbar(cax)

    # Are there a list of classes?
    if classes:
        labels = classes
    else:
        labels = np.arange(cm.shape[0])

    # Label the axes
    ax.set(
        title="Confusion Matrix",
        xlabel="Predicted label",
        ylabel="True label",
        xticks=np.arange(n_classes),  # create enough axis slots for each class
        yticks=np.arange(n_classes),
        xticklabels=labels,  # axes will labeled with class names (if they exist) or ints
        yticklabels=labels,
    )

    # Make x-axis labels appear on bottom
    ax.xaxis.set_label_position("bottom")
    ax.xaxis.tick_bottom()
    ### Added: Rotate xticks for readability & increase font size (required due to such a large confusion matrix)
    plt.xticks(rotation=90, fontsize=text_size)
    plt.yticks(fontsize=text_size)

    # Set the threshold for different colors
    threshold = (cm.max() + cm.min()) / 2.0

    # Plot the text on each cell
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if norm:
            plt.text(
                j,
                i,
                f"{cm[i, j]} ({cm_norm[i, j] * 100:.1f}%)",
                horizontalalignment="center",
                color="white" if cm[i, j] > threshold else "black",
                size=text_size,
            )
        else:
            plt.text(
                j,
                i,
                f"{cm[i, j]}",
                horizontalalignment="center",
                color="white" if cm[i, j] > threshold else "black",
                size=text_size,
            )

    # Save the figure to the current working directory
    if savefig:
        fig.savefig("confusion_matrix.png")


In [ ]:
make_confusion_matrix(y_test, pred, list(labels.values()))